We have provided two JSON datasets - one contains information on airport delays, and the other contains information on weather in cities. The goal of this take home question is to write code that will ingest these two datasets, join them together, and output a CSV which summarizes delays and weather for each city, by month. Once you have handled data ingestion & transformation, we'd like you to answer a few questions based on the data using code:

* Which city has the highest correlation between weather delays and rainfall?

* Which month has the highest correlation between weather delays and rainfall?

* See if you can find any other strong correlations between the climate data and airport delay data (does not have to be by rainfall, city, or month - whatever looks interesting to you!)

* Tell us your take-aways from this analysis

You are welcome to use any programming languages and frameworks that you feel will help you best solve this problem, just keep in mind that while the datasets we're providing you are static and fairly small, you should treat this as if you were creating a production-ready feature: we want to see your version of clean, reusable, scalable data processing code.

While working on your solution, please feel free to ask us any questions you might have in Slack - treat us as a remote team with a product owner who can clarify requirements, and engineering peers who you can bounce technical ideas off of.

In [1]:
## Un-comment to install packages

# !pip install pandas
# !pip install requests
# !pip install pandas_profiling


In [2]:
#packages used
import pandas as pd
from pprint import pprint
import requests
from pandas.io.json import json_normalize
import itertools
import pandas_profiling 

In [3]:
# getting the values from the "API" takes a couple seconds due to the size of the airport data and using requests.
# This could be streamlined with either a streaming/listening API or a direct API call.

climate_url = "https://s3.amazonaws.com/yapta-take-home/climate.json"
res = requests.get(climate_url)
climate = res.json()

airport_url = "https://s3.amazonaws.com/yapta-take-home/airport_delays.json"
res = requests.get(airport_url)
airport = res.json()

ConnectionError: HTTPSConnectionPool(host='s3.amazonaws.com', port=443): Max retries exceeded with url: /yapta-take-home/climate.json (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11236d550>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [ ]:
#viewing our climate data
pprint(climate[:2])

# Looking at the climate data, we can see we have a dataset with each month's average (in metric units) of:
* high temp (Celcius)
* low temp (Celcius)
* Drydays
* SnowDays
* Rainfall (In Centimeters)

In [ ]:
#viewing our airport data
pprint(airport[:2])

# Looking at the Airport_Delay data we can see:
* Airport -
    * Code: Airport Code
    * Name: Airport's City, State and Name of airport
* Carrier
    * Code (2-3 character code) e.g. AA
    * Name (Actual Name of Carrier) e.g. American Airlines Inc.
* Statistics -
    * Flights (status of flight and count of those statuses, Total of flights)
        * On Time
        * Total
        * Delayed
        * Diverted
    * Number of Delays (Reason for Delay and count of)
        * Late Aircraft
        * Weather 
        * Carrier
        * Security
        * National Aviation System
    * Minutes Delayed (Sum of minutes delayed per reason)
        * Late Aircraft
        * Weather 
        * Carrier
        * Security
        * National Aviation System
        * Total
* Time
    * Label (year/month)
    * Year
    * Month


## To get these 2 datasets to match up, the best way is to flatten out each nested dictionary and then join them on city and month using pandas

In [ ]:
climate_data = json_normalize(data=climate, record_path="monthlyAvg",
                             meta=['city','country','id']) #parses out the monthly weather information

seq = itertools.cycle([1, 2, 3,4,5,6,7,8,9,10,11,12])

climate_data['month'] = [next(seq) for count in range(climate_data.shape[0])] #adds a numbered month column 


In [ ]:
# checking for all US cities and how many are in the data set
mask = climate_data[climate_data['country'] == "United States"]
print(mask['city'].unique())
print(len(mask['city'].unique()))

In [ ]:
# gets the index range for Dallas TX to change it to Dallas/Forth Worth TX
climate_data[climate_data['city'] == "Dallas TX"].index

# EDA showed that Dallas TX doesnt match up with Dallas/FortWorth in the Airport Data
climate_data.at[288:300,'city'] = "Dallas/Fort Worth TX"

In [ ]:
# parses the airport data into a dataframe
airport_data = json_normalize(data=airport,sep="_")

In [ ]:
# checking our data came in correctly
airport_data.head(3)

### Feature Engineering for filtering and additional metrics

In [ ]:
# parses out city,state and airport name
airport_data[["city_state","name"]] = airport_data["airport_name"].str.split(":",expand=True) 

# parses state out of city column into it's own column
airport_data[["city",'state']] = airport_data['city_state'].str.split(",",expand=True) 

# creates a city_id column that can be used to match up with the climate data city column
airport_data['city_id'] = airport_data['city'] + airport_data['state'] 

# create column for country if data for european airports comes from another source
airport_data['country'] = "United States" 

# create column of total delays
airport_data['statistics_# of delays_total'] = (airport_data['statistics_# of delays_carrier'] +
                                               airport_data['statistics_# of delays_late aircraft'] + 
                                               airport_data['statistics_# of delays_national aviation system'] +
                                               airport_data['statistics_# of delays_security'] +  
                                               airport_data['statistics_# of delays_weather'])

# add hour columns for delays
airport_data['statistics_hours delayed_carrier'] = airport_data['statistics_minutes delayed_carrier'] / 60
airport_data['statistics_hours delayed_late aircraft'] = airport_data['statistics_minutes delayed_late aircraft'] / 60
airport_data['statistics_hours delayed_national aviation system'] = airport_data['statistics_minutes delayed_national aviation system'] / 60
airport_data['statistics_hours delayed_security'] = airport_data['statistics_minutes delayed_security'] / 60
airport_data['statistics_hours delayed_total'] = airport_data['statistics_minutes delayed_total'] / 60
airport_data['statistics_hours delayed_weather'] = airport_data['statistics_minutes delayed_weather'] / 60


In [ ]:
# round dataframe to 2 decimal points
airport_data = airport_data.round(2)

In [ ]:
# checking numer of cities in Airport Data
print(airport_data['city_id'].unique())
print(len(airport_data['city_id'].unique()))

In [ ]:
# inner merge airport data with climate data
all_info = pd.merge(left = airport_data, 
                    right= climate_data,
                    how = "inner",
                    left_on = ["city_id","time_month"],
                    right_on = ["city","month"],
                    suffixes=("_air","_cli")
                   )

In [ ]:
# removing redundant columns
all_info.drop(columns=['airport_name','country_cli','city_cli','city_air','time_month'],inplace=True) 

In [ ]:
# setting a multi level index allows for a cleaner view of the data 
# and fullfills the requirement for the data to be sorted by city and month

export = all_info.set_index(keys=["time_year","city_id",'month']).sort_index(axis=0,ascending=True)

In [ ]:
#exporting multi level index dataframes to CSV files
export.to_csv("./clean_airport_all.csv")

In [ ]:
# handy package that gives a statistical overview of all the data in the dataframe
pandas_profiling.ProfileReport(export)